In [1]:
import pandas as pd
import requests
import csv

In [2]:
import pandas as pd
canada = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)[0]


In [3]:
canada.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


To process cells which have an assigned value

In [4]:
canada=canada[canada.Borough != 'Not assigned']

In [5]:
canada.columns = ['Postcode', 'Borough', 'Neighbourhood']

To reset index after removing cells which have no assigned value

In [6]:
ncanada = canada.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()


To assign value to the neighborhood which has no value

In [7]:
for index, row in ncanada.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']


In [8]:
ncanada.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# CREATE A DATAFRAME OF LATITUDES AND LONGITUDES

READ THE LATITUDES AND LONGITUDES FROM THE URL AND MERGE THE TWO TABLES 

In [9]:

location= pd.read_csv("http://cocl.us/Geospatial_data")
location.columns=['Postcode', 'Latitude', 'Longitude']

location.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
lcanada  = pd.merge(ncanada, location, on='Postcode')
lcanada.head()
 

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [13]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


FIND THE LATITUDE AND LONGITUDES FOR TORONTO

In [14]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [15]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [16]:
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  39.49 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  28.88 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  36.15 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  43.62 MB/s


In [19]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(lcanada['Borough'].unique()),
        lcanada.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [20]:
central_data = lcanada[lcanada['Borough'] == 'Central Toronto'].reset_index(drop=True)
central_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
3,M4S,Central Toronto,Davisville,43.704324,-79.388790
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160


FIND LONGITUDE AND LATITUDES OF CENTRAL TORONTO

In [18]:
address = 'Central Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.653963, -79.387207.


CREATE MAP OF CENTRAL TORONTO

In [23]:
map_CentralToronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(central_data['Latitude'], central_data['Longitude'], central_data['Borough'], central_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_CentralToronto)  
    
map_CentralToronto


DEFINE FOURSQUARE CREDENTIALS AND VERSION

In [24]:
CLIENT_ID = 'YCTYSVQJVZPUNP5IOD0NWZVDRSMPA4M2EWUTJETCWVTRFPTI' # your Foursquare ID
CLIENT_SECRET = 'Z44A54OBQ5XT03V4U2FFI43ICW1F2N32PX0F0KG1MDSCTY2G' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YCTYSVQJVZPUNP5IOD0NWZVDRSMPA4M2EWUTJETCWVTRFPTI
CLIENT_SECRET:Z44A54OBQ5XT03V4U2FFI43ICW1F2N32PX0F0KG1MDSCTY2G


In [25]:
central_data.shape

(9, 5)

GET THE NEIGHBORHOOD NAME

In [26]:
central_data.loc[0, 'Neighbourhood']

'Lawrence Park'

 GET THE NEIGHBOURHOOD LATITUDE AND LONGITUDE

In [27]:
neighborhood_latitude = central_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = central_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = central_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Lawrence Park are 43.7280205, -79.3887901.


 The top 50 venues that are in Lawrence Park within a radius of 500 meters

Create the GET request URL. Name your URL url.

In [28]:
LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 


'https://api.foursquare.com/v2/venues/explore?&client_id=YCTYSVQJVZPUNP5IOD0NWZVDRSMPA4M2EWUTJETCWVTRFPTI&client_secret=Z44A54OBQ5XT03V4U2FFI43ICW1F2N32PX0F0KG1MDSCTY2G&v=20180605&ll=43.7280205,-79.3887901&radius=500&limit=50'

In [29]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cbfc3b94c1f67438fd5b964'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-50e6da19e4b0d8a78a0e9794-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d163941735',
         'name': 'Park',
         'pluralName': 'Parks',
         'primary': True,
         'shortName': 'Park'}],
       'id': '50e6da19e4b0d8a78a0e9794',
       'location': {'address': '3055 Yonge Street',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Lawrence Avenue East',
        'distance': 465,
        'formattedAddress': ['3055 Yonge Street (Lawrence Avenue East)',
         'Toronto ON',
         'Canada'],
        'labeledLatLngs': [{

THE INFORMATION IS IN THE ITEMS KEY SO WE HAVE TO CALL THE get_category_type FUNCTION FROM FOURSQUARE

In [30]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

 To clean the json and structure it into a pandas dataframe.

In [31]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Lawrence Park Ravine,Park,43.726963,-79.394382
1,Zodiac Swim School,Swim School,43.728532,-79.382860
2,TTC Bus #162 - Lawrence-Donway,Bus Line,43.728026,-79.382805


In [32]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


# EXPLORE NEIGHBOURHOODS OF CENTRAL TORONTO

 Create a function to repeat the same process to all the neighborhoods in CENTRAL TORONTO

In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
central_venues = getNearbyVenues(names=central_data['Neighbourhood'],
                                   latitudes=central_data['Latitude'],
                                   longitudes=central_data['Longitude']
                                  )


Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville


HOW MANY VENUES FOR EACH NEIGHBOURHOOD

In [52]:
print(central_venues.shape)
central_venues.head()

(119, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
2,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
3,Davisville North,43.712751,-79.390197,Sherwood Park,43.716551,-79.387776,Park
4,Davisville North,43.712751,-79.390197,Summerhill Market North,43.715499,-79.392881,Food & Drink Shop


In [55]:
central_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Davisville,37,37,37,37,37,37
Davisville North,11,11,11,11,11,11
"Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West",15,15,15,15,15,15
"Forest Hill North, Forest Hill West",4,4,4,4,4,4
Lawrence Park,3,3,3,3,3,3
"Moore Park, Summerhill East",4,4,4,4,4,4
North Toronto West,19,19,19,19,19,19
Roselawn,1,1,1,1,1,1
"The Annex, North Midtown, Yorkville",25,25,25,25,25,25


In [54]:
print('There are {} uniques categories.'.format(len(central_venues['Venue Category'].unique())))

There are 63 uniques categories.


# Analyze Each Neighborhood

In [56]:
# one hot encoding
central_onehot = pd.get_dummies(central_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
central_onehot['Neighborhood'] = central_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [central_onehot.columns[-1]] + list(central_onehot.columns[:-1])
central_onehot = central_onehot[fixed_columns]

central_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Dance Studio,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gift Shop,Gourmet Shop,Greek Restaurant,Gym,History Museum,Hotel,Indian Restaurant,Italian Restaurant,Jewelry Store,Jewish Restaurant,Light Rail Station,Liquor Store,Medical Center,Metro Station,Mexican Restaurant,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Lawrence Park,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,Lawrence Park,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Davisville North,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Davisville North,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [60]:
central_onehot.shape

(119, 64)

Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [61]:
central_grouped = central_onehot.groupby('Neighborhood').mean().reset_index()
central_grouped

,Neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bagel Shop,Breakfast Spot,Brewery,Burger Joint,Bus Line,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Convenience Store,Cosmetics Shop,Dance Studio,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint,Garden,Gift Shop,Gourmet Shop,Greek Restaurant,Gym,History Museum,Hotel,Indian Restaurant,Italian Restaurant,Jewelry Store,Jewish Restaurant,Light Rail Station,Liquor Store,Medical Center,Metro Station,Mexican Restaurant,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pub,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Spa,Sporting Goods Shop,Sports Bar,Supermarket,Sushi Restaurant,Swim School,Tennis Court,Thai Restaurant,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yoga Studio
0,Davisville,0.000000,0.027027,0.00,0.000000,0.000000,0.027027,0.027027,0.000000,0.054054,0.00,0.000000,0.000000,0.054054,0.000000,0.00,0.027027,0.081081,0.027027,0.027027,0.000000,0.027027,0.000000,0.027027,0.0,0.000000,0.027027,0.027027,0.027027,0.00,0.000000,0.027027,0.054054,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.027027,0.000000,0.027027,0.081081,0.00,0.000000,0.000000,0.054054,0.000000,0.081081,0.027027,0.000000,0.000000,0.000000,0.000000,0.054054,0.000000,0.027027,0.027027,0.027027,0.00,0.00,0.000000,0.000000
1,Davisville North,0.000000,0.090909,0.00,0.000000,0.090909,0.000000,0.090909,0.000000,0.000000,0.00,0.000000,0.090909,0.000000,0.000000,0.00,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.0,0.000000,0.000000,0.000000,0.090909,0.00,0.090909,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.090909,0.00,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000
2,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",0.066667,0.000000,0.00,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.133333,0.066667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.066667,0.00,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.00,0.133333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.066667,0.066667,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.066667,0.000000
3,"Forest Hill North, Forest Hill West",0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.25,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.25,0.00,0.000000,0.000000
4,Lawrence Park,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000
5,"Moore Park, Summerhill East",0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.250000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0

In [62]:
central_grouped.shape

(9, 64)

 Each neighborhood along with the top 5 most common venues

In [63]:
num_top_venues = 5

for hood in central_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = central_grouped[central_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
            venue  freq
0  Sandwich Place  0.08
1     Pizza Place  0.08
2    Dessert Shop  0.08
3            Café  0.05
4      Restaurant  0.05


----Davisville North----
               venue  freq
0                Gym  0.09
1              Hotel  0.09
2  Food & Drink Shop  0.09
3       Dance Studio  0.09
4        Pizza Place  0.09


----Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West----
                 venue  freq
0                  Pub  0.13
1          Coffee Shop  0.13
2  American Restaurant  0.07
3          Supermarket  0.07
4   Light Rail Station  0.07


----Forest Hill North, Forest Hill West----
              venue  freq
0              Park  0.25
1  Sushi Restaurant  0.25
2     Jewelry Store  0.25
3             Trail  0.25
4               Pub  0.00


----Lawrence Park----
                 venue  freq
0                 Park  0.33
1          Swim School  0.33
2             Bus Line  0.33
3  Rental Car Location  0.00
4                  Pub  0.0

In [64]:
import numpy as np

In [65]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create the new dataframe and display the top 10 venues for each neighborhood.

In [66]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = central_grouped['Neighborhood']

for ind in np.arange(central_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(central_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Davisville,Dessert Shop,Sandwich Place,Pizza Place,Italian Restaurant,Restaurant,Café,Coffee Shop,Sushi Restaurant,Park,Dance Studio
1,Davisville North,Sandwich Place,Asian Restaurant,Gym,Breakfast Spot,Dance Studio,Burger Joint,Hotel,Park,Pizza Place,Food & Drink Shop
2,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",Pub,Coffee Shop,American Restaurant,Sports Bar,Vietnamese Restaurant,Fried Chicken Joint,Light Rail Station,Medical Center,Pizza Place,Convenience Store
3,"Forest Hill North, Forest Hill West",Sushi Restaurant,Trail,Jewelry Store,Park,Food & Drink Shop,Diner,Farmers Market,Fast Food Restaurant,Flower Shop,Yoga Studio
4,Lawrence Park,Bus Line,Park,Swim School,Yoga Studio,Diner,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint


# Cluster Neighborhoods

In [67]:
kclusters = 5
central_grouped_clustering = central_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(central_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 4, 3, 0, 2, 1, 2], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [82]:
central_merged = central_data

central_merged['Cluster Labels'] = kmeans.labels_


In [83]:
central_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197,2
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,2
3,M4S,Central Toronto,Davisville,43.704324,-79.388790,4
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,3


In [90]:
central_merged = central_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

central_merged.head() 

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Bus Line,Park,Swim School,Yoga Studio,Diner,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint
1,M4P,Central Toronto,Davisville North,43.712751,-79.390197,2,Sandwich Place,Asian Restaurant,Gym,Breakfast Spot,Dance Studio,Burger Joint,Hotel,Park,Pizza Place,Food & Drink Shop
2,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,2,Coffee Shop,Sporting Goods Shop,Sandwich Place,Gift Shop,Fast Food Restaurant,Diner,Dessert Shop,Metro Station,Mexican Restaurant,Park
3,M4S,Central Toronto,Davisville,43.704324,-79.388790,4,Dessert Shop,Sandwich Place,Pizza Place,Italian Restaurant,Restaurant,Café,Coffee Shop,Sushi Restaurant,Park,Dance Studio
4,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,3,Restaurant,Gym,Playground,Tennis Court,Flower Shop,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Fried Chicken Joint


 Visualize the resulting clusters

In [92]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(central_merged['Latitude'], central_merged['Longitude'], central_merged['Neighbourhood'], central_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

CLUSTER 1

In [93]:
central_merged.loc[central_merged['Cluster Labels'] == 0, 
                   central_merged.columns[[1] + list(range(3, central_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Central Toronto,43.686412,-79.400049,0,Pub,Coffee Shop,American Restaurant,Sports Bar,Vietnamese Restaurant,Fried Chicken Joint,Light Rail Station,Medical Center,Pizza Place,Convenience Store


CLUSTER 2

In [94]:
central_merged.loc[central_merged['Cluster Labels'] == 1, 
                   central_merged.columns[[1] + list(range(0, central_merged.shape[1]))]]

,Borough,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Central Toronto,M5P,Central Toronto,"Forest Hill North, Forest Hill West",43.696948,-79.411307,1,Sushi Restaurant,Trail,Jewelry Store,Park,Food & Drink Shop,Diner,Farmers Market,Fast Food Restaurant,Flower Shop,Yoga Studio


CLUSTER 3

In [95]:
central_merged.loc[central_merged['Cluster Labels'] == 2, 
                   central_merged.columns[[1] + list(range(0, central_merged.shape[1]))]]

,Borough,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Bus Line,Park,Swim School,Yoga Studio,Diner,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Fried Chicken Joint
1,Central Toronto,M4P,Central Toronto,Davisville North,43.712751,-79.390197,2,Sandwich Place,Asian Restaurant,Gym,Breakfast Spot,Dance Studio,Burger Joint,Hotel,Park,Pizza Place,Food & Drink Shop
2,Central Toronto,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,2,Coffee Shop,Sporting Goods Shop,Sandwich Place,Gift Shop,Fast Food Restaurant,Diner,Dessert Shop,Metro Station,Mexican Restaurant,Park
6,Central Toronto,M5N,Central Toronto,Roselawn,43.711695,-79.416936,2,Garden,Yoga Studio,Dance Studio,History Museum,Gym,Greek Restaurant,Gourmet Shop,Gift Shop,Fried Chicken Joint,Food & Drink Shop
8,Central Toronto,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,2,Sandwich Place,Café,Coffee Shop,Pizza Place,American Restaurant,History Museum,Indian Restaurant,Jewish Restaurant,Liquor Store,Park


CLUSTER 4

In [126]:
central_merged.loc[central_merged['Cluster Labels'] == 3, 
                   central_merged.columns[[1] + list(range(5, central_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,3,Restaurant,Gym,Playground,Tennis Court,Flower Shop,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Fried Chicken Joint


CLUSTER 5

In [96]:
central_merged.loc[central_merged['Cluster Labels'] == 4, 
                   central_merged.columns[[1] + list(range(5, central_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Central Toronto,4,Dessert Shop,Sandwich Place,Pizza Place,Italian Restaurant,Restaurant,Café,Coffee Shop,Sushi Restaurant,Park,Dance Studio


# When we are exploring data in Central Toronto we have

LAWRENCE PARK has

1.Bus Line
2.Park
3.Swim School
4.Yoga studio

DAVISVILLE NORTH

1.GYM
2.DANCE STUDIO
3.PARK


# Exploring data for EAST TORONTO

In [21]:
East_data = lcanada[lcanada['Borough'] == 'East Toronto'].reset_index(drop=True)
East_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558


In [22]:
address = 'East Toronto, CA'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of East Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of East Toronto are 43.6247901, -79.3934918.


In [36]:
map_EastToronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(East_data['Latitude'], East_data['Longitude'], East_data['Borough'], East_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_EastToronto)  
    
map_EastToronto


In [38]:
East_data.loc[0, 'Neighbourhood']

'The Beaches'

In [40]:
neighborhood_latitude = East_data.loc[0, 'Latitude'] 
neighborhood_longitude = East_data.loc[0, 'Longitude']
neighborhood_name = East_data.loc[0, 'Neighbourhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [41]:
LIMIT = 50
radius = 500 
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url  

'https://api.foursquare.com/v2/venues/explore?&client_id=YCTYSVQJVZPUNP5IOD0NWZVDRSMPA4M2EWUTJETCWVTRFPTI&client_secret=Z44A54OBQ5XT03V4U2FFI43ICW1F2N32PX0F0KG1MDSCTY2G&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=50'

In [42]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cbfc69fdd57972412c454ef'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-4ad4c062f964a52011f820e3-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/food_grocery_',
          'suffix': '.png'},
         'id': '50aa9e744b90af0d42d5de0e',
         'name': 'Health Food Store',
         'pluralName': 'Health Food Stores',
         'primary': True,
         'shortName': 'Health Food Store'}],
       'id': '4ad4c062f964a52011f820e3',
       'location': {'address': '125 Southwood Dr',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 471,
        'formattedAddress': ['125 Southwood Dr',
         'Toronto ON M4E 0B8',
         'Canada'],
        'labeledLatLngs': [{'label': 'display',
      

In [43]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
1,Grover Pub and Grub,Pub,43.679181,-79.297215
2,Starbucks,Coffee Shop,43.678798,-79.298045
3,Upper Beaches,Neighborhood,43.680563,-79.292869


In [44]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [46]:
East_venues = getNearbyVenues(names=East_data['Neighbourhood'],
                                   latitudes=East_data['Latitude'],
                                   longitudes=East_data['Longitude']
                                  )



The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Business Reply Mail Processing Centre 969 Eastern


In [47]:
print(East_venues.shape)
East_venues.head()

(123, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


In [48]:
East_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
Studio District,38,38,38,38,38,38
The Beaches,4,4,4,4,4,4
"The Beaches West, India Bazaar",21,21,21,21,21,21
"The Danforth West, Riverdale",44,44,44,44,44,44


In [49]:
print('There are {} uniques categories.'.format(len(East_venues['Venue Category'].unique())))

There are 69 uniques categories.


In [52]:
East_onehot = pd.get_dummies(East_venues[['Venue Category']], prefix="", prefix_sep="")

East_onehot['Neighborhood'] = East_venues['Neighborhood'] 

fixed_columns = [East_onehot.columns[-1]] + list(East_onehot.columns[:-1])
East_onehot = East_onehot[fixed_columns]
East_onehot.head()

,Yoga Studio,American Restaurant,Auto Workshop,Bakery,Bank,Bar,Bookstore,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Light Rail Station,Liquor Store,Middle Eastern Restaurant,Movie Theater,Music Store,Neighborhood,Park,Pet Store,Pizza Place,Pub,Recording Studio,Restaurant,Sandwich Place,Seafood Restaurant,Skate Park,Smoke Shop,Spa,Sports Bar,Stationery Store,Steakhouse,Sushi Restaurant,Trail
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [53]:
East_onehot.shape

(123, 69)

In [54]:
East_grouped = East_onehot.groupby('Neighborhood').mean().reset_index()
East_grouped

,Neighborhood,Yoga Studio,American Restaurant,Auto Workshop,Bakery,Bank,Bar,Bookstore,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Café,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Dessert Shop,Diner,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Food & Drink Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gastropub,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Light Rail Station,Liquor Store,Middle Eastern Restaurant,Movie Theater,Music Store,Park,Pet Store,Pizza Place,Pub,Recording Studio,Restaurant,Sandwich Place,Seafood Restaurant,Skate Park,Smoke Shop,Spa,Sports Bar,Stationery Store,Steakhouse,Sushi Restaurant,Trail
0,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.0625,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,0.0625,0.000000,0.000000,0.000000,0.000000,0.062500,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.0625,0.062500,0.000000,0.000000,0.0625,0.0625,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000
1,Studio District,0.026316,0.052632,0.0000,0.052632,0.026316,0.026316,0.026316,0.026316,0.000000,0.000000,0.000000,0.105263,0.026316,0.000000,0.026316,0.026316,0.026316,0.078947,0.026316,0.0000,0.026316,0.000000,0.026316,0.000000,0.000000,0.0000,0.000000,0.000000,0.026316,0.000000,0.000000,0.000000,0.0000,0.0000,0.026316,0.000000,0.000000,0.000000,0.026316,0.00,0.026316,0.000000,0.000000,0.052632,0.000000,0.000000,0.026316,0.0000,0.000000,0.026316,0.000000,0.026316,0.026316,0.000000,0.000000,0.000000,0.0000,0.000000,0.026316,0.026316,0.0000,0.0000,0.026316,0.000000,0.026316,0.000000,0.000000,0.000000
2,The Beaches,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.0000,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,"The Beaches West, India Bazaar",0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.047619,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.047619,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.047619,0.047619,0.000000,0.047619,0.000000,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.047619,0.000000,0.00,0.047619,0.000000,0.047619,0.047619,0.000000,0.000000,0.000000,0.0000,0.047619,0.000000,0.047619,0.000000,0.047619,0.047619,0.047619,0.047619,0.0000,0.000000,0.095238,0.000000,0.0000,0.0000,0.000000,0.000000,0.000000,0.047619,0.047619,0.000000
4,"The Danforth West, Riverdale",0.022727,0.022727,0.0000,0.022727,0.000000,0.000000,0.045455,0.022727,0.022727,0.000000,0.000000,0.022727,0.000000,0.022727,0.000000,0.000000,0.000000,0.090909,0.000000,0.0000,0.000000,0.022727,0.000000,0.022727,0.022727,0.0000,0.000000,0.000000,0.000000,0.000000,0.022727,0.045455,0.0000,0.0000,0.000000,0.181818,0.022727,0.000000,0.000000,0.00,0.068182,0.022727,0.000000,0.045455,0.022727,0.022727,0.000000,0.0000,0.022727,0.000000,0.000000,0.000000,0.000000,0.000000,0.022727,0.022727,0.0000,0.022727,0.000000,0.000000,0.0000,

In [55]:
num_top_venues = 5

for hood in East_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = East_grouped[East_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0            Comic Shop  0.06
1         Burrito Place  0.06
2  Fast Food Restaurant  0.06
3    Light Rail Station  0.06
4  Gym / Fitness Center  0.06


----Studio District----
                 venue  freq
0                 Café  0.11
1          Coffee Shop  0.08
2               Bakery  0.05
3  American Restaurant  0.05
4   Italian Restaurant  0.05


----The Beaches----
                       venue  freq
0                Coffee Shop  0.25
1          Health Food Store  0.25
2                        Pub  0.25
3        Japanese Restaurant  0.00
4  Middle Eastern Restaurant  0.00


----The Beaches West, India Bazaar----
            venue  freq
0  Sandwich Place  0.10
1       Pet Store  0.05
2             Pub  0.05
3  Ice Cream Shop  0.05
4    Intersection  0.05


----The Danforth West, Riverdale----
                    venue  freq
0        Greek Restaurant  0.18
1             Coffee Shop  0.09
2         

In [62]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [63]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = East_grouped['Neighborhood']

for ind in np.arange(East_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(East_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Business Reply Mail Processing Centre 969 Eastern,Garden Center,Gym / Fitness Center,Light Rail Station,Brewery,Pizza Place,Comic Shop,Recording Studio,Restaurant,Burrito Place,Farmers Market
1,Studio District,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Ice Cream Shop,Latin American Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store
2,The Beaches,Coffee Shop,Pub,Health Food Store,Dessert Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Trail
3,"The Beaches West, India Bazaar",Sandwich Place,Fast Food Restaurant,Pizza Place,Coffee Shop,Intersection,Italian Restaurant,Gym,Liquor Store,Burrito Place,Movie Theater
4,"The Danforth West, Riverdale",Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Trail,Diner,Indian Restaurant,Grocery Store


In [72]:
kclusters = 3

East_grouped_clustering = East_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(East_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 1, 0, 0], dtype=int32)

In [73]:
East_merged = East_data

East_merged['Cluster Labels'] = kmeans.labels_


East_merged = East_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

East_merged.head() 

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Coffee Shop,Pub,Health Food Store,Dessert Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Trail
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Trail,Diner,Indian Restaurant,Grocery Store
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,1,Sandwich Place,Fast Food Restaurant,Pizza Place,Coffee Shop,Intersection,Italian Restaurant,Gym,Liquor Store,Burrito Place,Movie Theater
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Ice Cream Shop,Latin American Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store
4,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,0,Garden Center,Gym / Fitness Center,Light Rail Station,Brewery,Pizza Place,Comic Shop,Recording Studio,Restaurant,Burrito Place,Farmers Market


In [74]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(East_merged['Latitude'], East_merged['Longitude'], East_merged['Neighbourhood'], East_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [75]:
East_merged.loc[East_merged['Cluster Labels'] == 0, East_merged.columns[[1] + list(range(5, East_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Trail,Diner,Indian Restaurant,Grocery Store
3,East Toronto,0,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Ice Cream Shop,Latin American Restaurant,Cheese Shop,Chinese Restaurant,Clothing Store
4,East Toronto,0,Garden Center,Gym / Fitness Center,Light Rail Station,Brewery,Pizza Place,Comic Shop,Recording Studio,Restaurant,Burrito Place,Farmers Market


In [76]:
East_merged.loc[East_merged['Cluster Labels'] == 1, East_merged.columns[[1] + list(range(5, East_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,East Toronto,1,Sandwich Place,Fast Food Restaurant,Pizza Place,Coffee Shop,Intersection,Italian Restaurant,Gym,Liquor Store,Burrito Place,Movie Theater


In [77]:
East_merged.loc[East_merged['Cluster Labels'] == 2, East_merged.columns[[1] + list(range(5, East_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,2,Coffee Shop,Pub,Health Food Store,Dessert Shop,Comfort Food Restaurant,Comic Shop,Convenience Store,Cosmetics Shop,Coworking Space,Trail


When we examine the data in East Toronto we see a lot of restaurants and stores and other shops

but no music or dance studios that are ideal to open the Non-Profit-Organization

When we are exploring data in Central Toronto we have
LAWRENCE PARK has

1.Bus Line 2.Park 3.Swim School 4.Yoga studio

DAVISVILLE NORTH

1.GYM 2.DANCE STUDIO 3.PARK

# CONCLUSION

The best first best place to open a Non-profit-Organization will be in DAVISVILLE NORTH IN CENTRAL TORONTO as it has 

GYM

DANCE STUDIO

PARK
 
few RESTAURANTS



The second best place to open a Non-profit-Organization will be in LAWRENCE PARK IN CENTRAL TORONTO as it has 

a BUS line for easy transportation,

SWIM SCHOOL

YOGA STUDIO

FEW REASTAURANTS